# Deep Learning for Regression

## The Dataset
[California Housing Dataset](https://developers.google.com/machine-learning/crash-course/california-housing-data-description).

## Import relevant modules

The following hidden code cell imports the necessary code to run the code in the rest of this Colaboratory.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import seaborn as sns


## Load the dataset

We use the California Housing Dataset.  The following code cell loads the separate .csv files and creates the following two pandas DataFrames:

* `train_df`, which contains the training set
* `test_df`, which contains the test set
   

In [ ]:
train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

# shuffle the examples
train_df = train_df.reindex(np.random.permutation(train_df.index)) 

test_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")

print(train_df.shape, test_df.shape)

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

## Normalize values

When building a model with multiple features, the values of each feature should cover roughly the same range.  The following code cell normalizes datasets by converting each raw value to its [Z-score](https://en.wikipedia.org/wiki/Standard_score).

In [ ]:
#@title Convert raw values to their Z-scores 

# Calculate the Z-scores of each column in the training set:
train_df_mean = train_df.mean()
train_df_std = train_df.std()
train_df_norm = (train_df - train_df_mean)/train_df_std

# Calculate the Z-scores of each column in the test set.
# test_df_mean = test_df.mean()
# test_df_std = test_df.std()
test_df_norm = (test_df - train_df_mean)/train_df_std

In [ ]:
train_df_norm.describe()

In [ ]:
transformed_train_df = pd.concat([train_df[['longitude', 'latitude']], train_df_norm.drop(['latitude', 'longitude'], axis=1)], axis=1)
transformed_train_df.head()

In [ ]:
transformed_test_df = pd.concat([test_df[['longitude', 'latitude']], test_df_norm.drop(['latitude', 'longitude'], axis=1)], axis=1)
transformed_test_df.head()

In [ ]:
transformed_train_df.shape, transformed_test_df.shape

## Feature Engineering

The following code cell creates a feature layer containing three features:

* `latitude` X `longitude` (a feature cross)
* `housing_median_age`
* `total_rooms`
* `total_bedrooms`
* `population`
* `households`
* `median_income`


This code cell specifies the features that you'll ultimately train the model on and how each of those features will be represented. The transformations (collected in `my_feature_layer`) don't actually get applied until you pass a DataFrame to it, which will happen when we train the model. 

In [ ]:
# Create an empty list that will eventually hold all created feature columns.
feature_columns = []

step_lat = max(transformed_train_df['latitude']) - min(transformed_train_df['latitude'])

# Create a bucket feature column for latitude.
latitude_as_a_numeric_column = tf.feature_column.numeric_column("latitude")
latitude_boundaries = list(np.arange(min(transformed_train_df['latitude']), 
                                     max(transformed_train_df['latitude']), 
                                     step_lat / 10.0))
latitude = tf.feature_column.bucketized_column(latitude_as_a_numeric_column, 
                                               latitude_boundaries)

step_lgn = max(transformed_train_df['longitude']) - min(transformed_train_df['longitude'])

# Create a bucket feature column for longitude.
longitude_as_a_numeric_column = tf.feature_column.numeric_column("longitude")
longitude_boundaries = list(np.arange(min(transformed_train_df['longitude']), 
                                      max(transformed_train_df['longitude']), 
                                      step_lgn / 10.0))

longitude = tf.feature_column.bucketized_column(longitude_as_a_numeric_column, 
                                                longitude_boundaries)

# Create a feature cross of latitude and longitude.
latitude_x_longitude = tf.feature_column.crossed_column([latitude, longitude], 
                                                        hash_bucket_size=100)

crossed_feature = tf.feature_column.indicator_column(latitude_x_longitude)
feature_columns.append(crossed_feature)  

# Represent housing_median_age as a floating-point value.
housing_median_age = tf.feature_column.numeric_column("housing_median_age")
feature_columns.append(housing_median_age)

# Represent total_rooms as a floating-point value.
total_rooms = tf.feature_column.numeric_column("total_rooms")
feature_columns.append(total_rooms)

# Represent total_bedrooms as a floating-point value.
total_bedrooms = tf.feature_column.numeric_column("total_bedrooms")
feature_columns.append(total_bedrooms)

# Represent population as a floating-point value.
population = tf.feature_column.numeric_column("population")
feature_columns.append(population)

# Represent households as a floating-point value.
households = tf.feature_column.numeric_column("households")
feature_columns.append(households)

# Represent median_income as a floating-point value.
median_income = tf.feature_column.numeric_column("median_income")
feature_columns.append(median_income)

# Convert the list of feature columns into a layer that will later be fed into
# the model. 
my_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

## Build a linear regression model as a baseline

Before creating a deep neural net, find a baseline loss by running a simple linear regression model that uses the feature layer you just created. 


In [ ]:
#@title Define the plotting function.

def plot_the_loss_curve(mse_train, mse_val):
  """Plot a curve of loss vs. epoch."""

  plt.figure(figsize=(10, 8))
  plt.xlabel("Epoch")
  plt.ylabel("Mean Squared Error")

  plt.plot(mse_train, label="Training Loss")
  plt.plot(mse_val, label="Validation Loss")
  plt.legend()
  # plt.ylim([mse.min()*0.95, mse.max() * 1.03])
  plt.show()  

In [ ]:
#@title Define functions to create and train a linear regression model
def create_model_linear(my_learning_rate, feature_layer):
  """Create and compile a simple linear regression model."""

  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add the layer containing the feature columns to the model.
  model.add(feature_layer)

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1))

  # Construct the layers into a model that TensorFlow can execute.
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error", 
                metrics=[tf.keras.metrics.MeanSquaredError()])
  return model           

## Define a training function

The `train_model` function trains the model from the input features and labels. The [tf.keras.Model.fit](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit) method performs the actual training. The following implementation passes a Python dictionary in which:

* The *keys* are the names of each feature (for example, `longitude`, `latitude`, and so on).
* The *value* of each key is a NumPy array containing the values of that feature. 

**Note:** Although you are passing *every* feature to `model.fit`, most of those values will be ignored. Only the features accessed by `my_feature_layer` will actually be used to train the model.

In [ ]:
def train_model(model, dataset, epochs, batch_size, label_name):
  """Feed a dataset into the model in order to train it."""

  # Split the dataset into features and label.
  features = {name:np.array(value) for name, value in dataset.items()}
  label = np.array(features.pop(label_name))
  history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, validation_split=0.2, shuffle=True)

  # Get details that will be useful for plotting the loss curve.
  mse_train = history.history['loss']
  mse_val = history.history['val_loss']

  return mse_train, mse_val

Run the following code cell to invoke the the functions defined in the preceding two code cells.

**Note:** Because we've scaled all the input data, **including the label**, the resulting loss values will be smaller.

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
number_epochs = 20
batch_size = 256
label_name = "median_house_value"

# Establish the model's topography.
model_lr = create_model_linear(learning_rate, my_feature_layer)

# Train the model on the normalized training set.
mse_train_lr, mse_val_lr = train_model(model_lr, transformed_train_df, number_epochs, batch_size, label_name)
plot_the_loss_curve(mse_train_lr, mse_val_lr)

In [ ]:
test_features = {name:np.array(value) for name, value in transformed_test_df.items()}
test_label = np.array(test_features.pop(label_name)) # isolate the label

print("\n Evaluate the linear regression model against the test set:")
result = model_lr.evaluate(x=test_features, y=test_label, batch_size=batch_size)

for item in zip(model_lr.metrics_names, result):
  print (item[0], item[1])

In [ ]:
model_lr.summary()

## Define a deep neural net model

The `create_model_deep` function defines the topography of the deep neural net, specifying the following:

* The number of layers in the deep neural net.
* The number of nodes in each layer.

The `create_model` function also defines the activation function of each layer.

In [ ]:
def create_model_deep(my_learning_rate, my_feature_layer, layers=[20, 12]):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add the layer containing the feature columns to the model.
  model.add(my_feature_layer)

  # Describe the topography of the model by calling the tf.keras.layers.Dense
  # method once for each layer. We've specified the following arguments:
  #   * units specifies the number of nodes in this layer.
  #   * activation specifies the activation function (Rectified Linear Unit).
  #   * name is just a string that can be useful when debugging.

  # Define the hidden layers
  for index, layer in enumerate(layers):
    model.add(tf.keras.layers.Dense(units=layer, 
                                    activation='relu', 
                                    name=f'Hidden{index}'))
  
  # Define the output layer.
  model.add(tf.keras.layers.Dense(units=1, name='Output'))                              
  
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])
  return model

## Call the functions to build and train a deep neural net


In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01

# Specify the label
label_name = "median_house_value"

# Establish the model's topography.
model_deep = create_model_deep(learning_rate, my_feature_layer, [20, 12])

# Train the model on the normalized training set. We're passing the entire
# normalized training set, but the model will only use the features
# defined by the feature_layer.

mse_train_deep, mse_val_deep = train_model(model_deep, transformed_train_df, number_epochs, batch_size, label_name)
plot_the_loss_curve(mse_train_deep, mse_val_deep)

In [ ]:
# After building a model against the training set, test that model
# against the test set.
result = model_deep.evaluate(x=test_features, 
                  y=test_label, 
                  batch_size=batch_size)

for item in zip(model_deep.metrics_names, result):
  print (item[0], item[1])

In [ ]:
model_deep.summary()

## Compare the two models

How did the deep neural net perform against the baseline linear regression model?

In [ ]:
plt.figure(figsize=(10, 8))
plt.xlabel("Epoch")
plt.ylabel("Mean Squared Error")

plt.plot(mse_train_lr, label="(Training Loss) Linear Model")
plt.plot(mse_train_deep, label="(Training Loss) Deep Model")
plt.plot(mse_val_lr, label="(Validation Loss) Linear Model")
plt.plot(mse_val_deep, label="(Validation Loss) Deep Model")
plt.legend()
# plt.ylim([mse.min()*0.95, mse.max() * 1.03])
plt.xticks(range(21))
plt.show() 

## Optimize the deep neural network's topography

Experiment with the number of layers of the deep neural network and the number of nodes in each layer.


In [ ]:
# Establish the model's topography.
model_deep_better = create_model_deep(learning_rate, my_feature_layer, [10, 8])

# Train the model on the normalized training set. We're passing the entire
# normalized training set, but the model will only use the features
# defined by the feature_layer.

mse_train_deep1, mse_val_deep1 = train_model(model_deep_better, transformed_train_df, number_epochs, batch_size, label_name)
plot_the_loss_curve(mse_train_deep1, mse_val_deep1)


In [ ]:
result = model_deep_better.evaluate(x=test_features, 
                  y=test_label, 
                  batch_size=batch_size)

for item in zip(model_deep.metrics_names, result):
  print (item[0], item[1])

In [ ]:
plt.figure(figsize=(10, 8))
plt.xlabel("Epoch")
plt.ylabel("Mean Squared Error")

plt.plot(mse_train_deep, label="(Training Loss) Deep Model")
plt.plot(mse_train_deep1, label="(Training Loss) Modified Deep Model")
plt.plot(mse_val_deep, label="(Validation Loss) Deep Model")
plt.plot(mse_val_deep1, label="(Validation Loss) Modified Deep Model")
plt.legend()
# plt.ylim([mse.min()*0.95, mse.max() * 1.03])
plt.xticks(range(21))
plt.show() 

## Regularize the deep neural network 

Notice that the model's loss against the test set is *higher* than the loss against the training set.  In other words, the deep neural network is *overfitting* to the data in the training set.  To reduce overfitting, regularize the model.  The course has suggested several different ways to regularize a model, including:

  * *L1 regularization*
  * *L2 regularization*
  * *Dropout regularization*

Your task is to experiment with one or more regularization mechanisms to bring the test loss closer to the training loss (while still keeping test loss relatively low).  

**Note:** When you add a regularization function to a model, you might need to tweak other hyperparameters. 

### Implementing L1 or L2 regularization

To use L1 or L2 regularization on a hidden layer, specify the `kernel_regularizer` argument to [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense). Assign one of the following methods to this argument:

* `tf.keras.regularizers.l1` for L1 regularization
* `tf.keras.regularizers.l2` for L2 regularization

Each of the preceding methods takes an `l` parameter, which adjusts the *regularization rate*. Assign a decimal value between 0 and 1.0 to `l`; the higher the decimal, the greater the regularization. For example, the following applies L2 regularization at a strength of 0.05. 

```
model.add(tf.keras.layers.Dense(units=20, 
                                activation='relu',
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                                name='Hidden1'))
```

### Implementing Dropout regularization

You implement dropout regularization as a separate layer in the topography. For example, the following code demonstrates how to add a dropout regularization layer between the first hidden layer and the second hidden layer:

```
model.add(tf.keras.layers.Dense( *define first hidden layer*)
 
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense( *define second hidden layer*)
```

The `rate` parameter to [tf.keras.layers.Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) specifies the fraction of nodes that the model should drop out during training. 


In [ ]:
# Use L2 Regularization

def create_model_deep_L2(my_learning_rate, my_feature_layer, layers=[20, 12], C=0.04):
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add the layer containing the feature columns to the model.
  model.add(my_feature_layer)

  # Describe the topography of the model by calling the tf.keras.layers.Dense
  # method once for each layer. We've specified the following arguments:
  #   * units specifies the number of nodes in this layer.
  #   * activation specifies the activation function (Rectified Linear Unit).
  #   * name is just a string that can be useful when debugging.

  # Define the hidden layers
  for index, layer in enumerate(layers):
    model.add(tf.keras.layers.Dense(units=layer, 
                                    activation='relu', 
                                    kernel_regularizer=tf.keras.regularizers.l2(C),
                                    name=f'Hidden{index}'))  
  # Define the output layer.
  model.add(tf.keras.layers.Dense(units=1,  
                                  name='Output'))                              
  
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])
  return model

In [ ]:
# Call the new create_model function and the other (unchanged) functions.

# The following variables are the hyperparameters.
label_name = "median_house_value"

# Establish the model's topography.
model_reg = create_model_deep_L2(learning_rate, my_feature_layer, [20, 10, 10, 8], C=0.001)

# Train the model on the normalized training set.
mse_train_reg, mse_val_reg = train_model(model_reg, transformed_train_df, number_epochs, batch_size, label_name)
plot_the_loss_curve(mse_train_reg, mse_val_reg)

In [ ]:
result = model_reg.evaluate(x=test_features, 
                  y=test_label, 
                  batch_size=batch_size)

for item in zip(model_reg.metrics_names, result):
  print (item[0], item[1])

# Deep Learning for Multi-Class Classification

## The Dataset
  
This MNIST dataset contains a lot of examples:

* The MNIST training set contains 60,000 examples.
* The MNIST test set contains 10,000 examples.

Each example contains a pixel map showing how a person wrote a digit. For example, the following images shows how a person wrote the digit `1` and how that digit might be represented in a 14x14 pixel map (after the input data is normalized). 

![Two images. The first image shows a somewhat fuzzy digit one. The second image shows a 14x14 floating-point array in which most of the cells contain 0 but a few cells contain values between 0.0 and 1.0. The pattern of nonzero values corresponds to the image of the fuzzy digit in the first image.](https://www.tensorflow.org/images/MNIST-Matrix.png)

Each example in the MNIST dataset consists of:

* A label specified by a annotator. Each label must be an integer from 0 to 9.  For example, in the preceding image, the rater would almost certainly assign the label `1` to the example.
* A 28x28 pixel map, where each pixel is an integer between 0 and 255. The pixel values are on a gray scale in which 0 represents white, 255 represents black, and values between 0 and 255 represent various shades of gray.  

This is a multi-class classification problem with 10 output classes, one for each digit.

## Load the dataset

`tf.keras` provides a set of convenience functions for loading well-known datasets. Each of these convenience functions does the following:

* Loads both the training set and the test set.
* Separates each set into features and labels.

The relevant convenience function for MNIST is called `mnist.load_data()`:

In [ ]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()


Notice that `mnist.load_data()` returned four separate values:

* `x_train` contains the training set's features.
* `y_train` contains the training set's labels.
* `x_test` contains the test set's features.
* `y_test` contains the test set's labels.

**Note:** The MNIST .csv training set is already shuffled.

## View the dataset

The .csv file for MNIST does not contain column names. Instead of column names, you use ordinal numbers to access different subsets of the MNIST dataset. In fact, it is probably best to think of `x_train` and `x_test` as three-dimensional NumPy arrays:  



In [ ]:
# Output example #2917 of the training set.
x_train[2917]

Alternatively, you can call matplotlib.pyplot.imshow to interpret the preceding numeric array as an image.



In [ ]:
# Use false colors to visualize the array.
plt.imshow(x_train[2917])

## Normalize feature values

The following code cell maps each feature value from its current representation (an integer between 0 and 255) to a floating-point value between 0 and 1.0. Store the floating-point values in `x_train_normalized` and `x_test_normalized`.

In [ ]:
x_train_normalized = x_train / 255.0
x_test_normalized = x_test / 255.0
print(x_train_normalized[2900][12]) # Output a normalized row

## Define a plotting function

The following function plots an accuracy curve:

In [ ]:
def plot_curve(hist):
  """Plot a curve of one or more classification metrics vs. epoch."""  
  # list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

  epochs = hist.epoch
  f, ax = plt.subplots(ncols=2, figsize=(20,8))
  ax[0].plot(epochs, hist.history['loss'], label='Training Loss')
  ax[0].plot(epochs, hist.history['val_loss'], label='Validation Loss')
  ax[0].set_xlabel('Epochs')
  ax[0].set_ylabel('Loss')
  ax[0].legend()
  ax[1].plot(epochs, hist.history['accuracy'], label='Training Accuracy')
  ax[1].plot(epochs, hist.history['val_accuracy'], label='Validation Accuracy')
  ax[1].set_xlabel('Epochs')
  ax[1].set_ylabel('Accuracy')
  ax[1].legend()


## Create a deep neural net model

The `create_model` function defines the topography of the deep neural net, specifying the following:

* The number of layers in the deep neural net.
* The number of nodes in each layer.
* Any regularization layers.

The `create_model` function also defines the *activation function* of each layer.  The activation function of the output layer is *softmax*, which will yield 10 different outputs for each example. Each of the 10 outputs provides the probability that the input example is a certain digit.


In [ ]:
def create_model(my_learning_rate):
  """Create and compile a deep neural net."""
  
  # All models in this course are sequential.
  model = tf.keras.models.Sequential()

  # The features are stored in a two-dimensional 28X28 array. 
  # Flatten that two-dimensional array into a a one-dimensional 
  # 784-element array.
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=32, activation='relu'))
  
  # Define a dropout regularization layer. 
  model.add(tf.keras.layers.Dropout(rate=0.2))

  # Define the output layer. The units parameter is set to 10 because
  # the model must choose among 10 possible output values (representing
  # the digits from 0 to 9, inclusive).
  #
  # Don't change this layer.
  model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
                           
  # Construct the layers into a model that TensorFlow can execute.  
  # Notice that the loss function for multi-class classification
  # is different than the loss function for binary classification.  
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  
  return model    


def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):
  """Train the model by feeding it data."""

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
  
  return history    

## Invoke the previous functions

Run the following code cell to invoke the preceding functions and actually train the model on the training set. 

**Note:** Due to several factors (for example, more examples and a more complex neural network) training MNIST might take longer than training the California Housing Dataset. Be patient.

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.003
epochs = 50
batch_size = 4000
validation_split = 0.2

# Establish the model's topography.
my_model = create_model(learning_rate)

# Train the model on the normalized training set.
hist = train_model(my_model, x_train_normalized, y_train, 
                           epochs, batch_size, validation_split)

In [ ]:
# Plot a graph of the metric vs. epochs.
plot_curve(hist)

In [ ]:
# Evaluate against the test set.
print("\n Evaluate the new model against the test set:")
result = my_model.evaluate(x=x_test_normalized, y=y_test, batch_size=batch_size)

for item in zip(my_model.metrics_names, result):
  print (item[0], item[1])

## Optimize the model

Can we reach at least 98% accuracy against the test set? 

In [ ]:
# We can reach 98% test accuracy with the 
# following configuration:
#   * One hidden layer of 256 nodes; no second 
#      hidden layer.
#   * dropout regularization rate of 0.4

# We can reach 98.2% test accuracy with the 
# following configuration:
#   * First hidden layer of 256 nodes; 
#     second hidden layer of 128 nodes.
#   * dropout regularization rate of 0.2

def create_better_model(my_learning_rate, layers=[256], dropout_rate=0.2):
  """Create and compile a deep neural net."""
  
  # All models in this course are sequential.
  model = tf.keras.models.Sequential()

  # The features are stored in a two-dimensional 28X28 array. 
  # Flatten that two-dimensional array into a a one-dimensional 
  # 784-element array.
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  
  for layer in layers:
    model.add(tf.keras.layers.Dense(units=layer, activation='relu'))
  
    # Define a dropout regularization layer. 
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))

  # Define the output layer. The units parameter is set to 10 because
  # the model must choose among 10 possible output values (representing
  # the digits from 0 to 9, inclusive).
  #
  # Don't change this layer.
  model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
                           
  # Construct the layers into a model that TensorFlow can execute.  
  # Notice that the loss function for multi-class classification
  # is different than the loss function for binary classification.  
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  
  return model  

In [ ]:
# Establish the model's topography.
my_model_better = create_better_model(learning_rate, layers=[256, 128, 64], dropout_rate=0.2)

# Train the model on the normalized training set.
hist = train_model(my_model_better, x_train_normalized, y_train, 
                           epochs, batch_size, validation_split)

In [ ]:
# Evaluate against the test set.
print("\n Evaluate the new model against the test set:")
result = my_model_better.evaluate(x=x_test_normalized, y=y_test, batch_size=batch_size)

for item in zip(my_model_better.metrics_names, result):
  print (item[0], item[1])